In [1]:
import torch
import numpy as np
from torch import nn, optim
from torch.distributions import Normal, Bernoulli
import matplotlib.pyplot as plt

In [2]:
LOGPI = np.log(np.pi)

In [3]:
class HarmonicTrialFunction(nn.Module):
    def __init__(self, alpha):
        super(HarmonicTrialFunction, self).__init__()
        self.alpha = nn.Parameter(alpha)
    
    def forward(self, x):
        # outputs logprob
        # 2.0 * because it's |\Psi|^2
        return 2.0 * ( 0.5*torch.log(self.alpha) - 0.25*LOGPI - 0.5*x*x*self.alpha*self.alpha )

    def local_energy(self, x):
        return self.alpha*self.alpha + (x*x) * (1.0 - self.alpha**4.0)

$$\log |\Psi(x)|^2$$

In [4]:
def true_mean_energy(alpha):
    return ((alpha**2)/2) + (1.0/(2*(alpha**2)))
def true_variance(alpha):
    return ((alpha**4 - 1)**2)/(2*alpha**4)

In [5]:
def normal_proposal(old_point):
    # symmetric
    return Normal(old_point, 0.3*torch.ones_like(old_point)).sample()

In [49]:
def symmetric_mh_acceptance_ratio(logprob, old_config, new_config):
    logacc = torch.min(torch.tensor(0.0), logprob(new_config) - logprob(old_config))
    return torch.exp(logacc)

def asymmetric_mh_acceptance_ratio(logprob, new_old_logprob, old_new_logprob, old_config, new_config):
    logacc = torch.min(torch.tensor(0.0), logprob(new_config) + 
                       old_new_logprob
                       - logprob(old_config)
                      - new_old_logprob)
    return torch.exp(logacc)

In [50]:
# def metropolis(trialfunc, proposal,num_steps=100):
#     config = torch.zeros(1)
#     all_configs = []
#     for step in range(num_steps):
#         next_config = proposal(config)
#         with torch.no_grad():
#             acc = mh_acceptance_ratio(trialfunc, config, next_config)
#         rand_val = np.random.rand()
#         if rand_val <= acc:
#             config = next_config
#         all_configs.append(config.clone()) # can we skip clone here?
#     return torch.stack(all_configs)

In [51]:
def metropolis_symmetric(trialfunc, proposal, num_walkers=2,num_steps=100):
    # with more walkers
    # design choice: walkers are always the batch dim
    config = torch.zeros(num_walkers, 1)
    all_configs = []
    for step in range(num_steps):
        next_config = proposal(config)
        with torch.no_grad():
            acc = symmetric_mh_acceptance_ratio(trialfunc, config, next_config)
        # acc shape should be (num_walkers, 1)
            accept_or_reject = Bernoulli(acc).sample() # accept is 1, reject is 0
            config = accept_or_reject*next_config + (1.0 - accept_or_reject)*config
            all_configs.append(config.clone()) # can we skip clone here?
    return torch.stack(all_configs, dim=1) # dim=1 to make walkers be the batch dim

In [52]:
def metropolis_asymmetric(trialfunc, proposal, num_walkers=2,num_steps=100):
    # with more walkers
    # design choice: walkers are always the batch dim
    config = torch.zeros(num_walkers, 1)
    all_configs = []
    for step in range(num_steps):
        next_config, next_current_logprob, current_next_logprob = proposal(config)
        with torch.no_grad():
            acc = asymmetric_mh_acceptance_ratio(trialfunc, 
                                                 next_current_logprob, 
                                                 current_next_logprob,
                                                 config, next_config)
            # acc shape should be (num_walkers, 1)
            accept_or_reject = Bernoulli(acc).sample() # accept is 1, reject is 0
            config = accept_or_reject*next_config + (1.0 - accept_or_reject)*config
            all_configs.append(config.clone()) # can we skip clone here?
    return torch.stack(all_configs, dim=1) # dim=1 to make walkers be the batch dim

In [53]:
def unadjusted_langevin(trialfunc, num_walkers=2, num_steps=100, eta=0.01):
    # seems hard to get this to converge
    config = torch.zeros(num_walkers, 1, requires_grad=True)
    grad_out = torch.ones_like(config, requires_grad=False)
    brownian_dist = Normal(torch.zeros_like(config, requires_grad=False),
                          torch.ones_like(config, requires_grad=False))
    all_configs = []
    for step in range(num_steps):
        # next config is from Langevin proposal (grad of logprob + gaussian noise)
        curr_config_logprobs = trialfunc(config)
        grads, = torch.autograd.grad(curr_config_logprobs,
                                    config,
                                    grad_outputs=grad_out,
                                    retain_graph=False)
        with torch.no_grad():
            next_config = config + eta*grads + np.sqrt(2.0*eta)*brownian_dist.sample()
        # then just append
        next_config.requires_grad_(True)
        all_configs.append(next_config)
        config = next_config
    return torch.stack(all_configs, dim=1)

In [54]:
def unadjusted_langevin(trialfunc, num_walkers=2, num_steps=100, eta=0.01):
    # seems hard to get this to converge
    config = torch.zeros(num_walkers, 1, requires_grad=True)
    grad_out = torch.ones_like(config, requires_grad=False)
    all_configs = []
    for step in range(num_steps):
        # next config is from Langevin proposal (grad of logprob + gaussian noise)
        curr_config_logprobs = trialfunc(config)
        grads, = torch.autograd.grad(curr_config_logprobs,
                                    config,
                                    grad_outputs=grad_out,
                                    retain_graph=False)
        with torch.no_grad():
            propdist = Normal(config + eta*grads, np.sqrt(2.0*eta))
            next_config = propdist.sample()
        # then just append
        next_config.requires_grad_(True)
        all_configs.append(next_config)
        config = next_config
    return torch.stack(all_configs, dim=1)

In [55]:
def mala(trialfunc, num_walkers=2, num_steps=100, eta=0.01):
    # this isn't right -- we need different MH filter for asymmetric proposal
    config = torch.zeros(num_walkers, 1)
    grad_out = torch.ones_like(config, requires_grad=False)
    brownian_dist = Normal(torch.zeros_like(config, requires_grad=False),
                          torch.ones_like(config, requires_grad=False))


    def mala_proposal(old_point):
        # 
        old_point.requires_grad_(True)
        curr_config_logprobs = trialfunc(old_point)
        grads, = torch.autograd.grad(curr_config_logprobs,
                                    old_point,
                                    grad_outputs=grad_out,
                                    retain_graph=False)
        with torch.no_grad():
            propdist = Normal(old_point + eta*grads, np.sqrt(2.0*eta))
            next_config = propdist.sample()
            new_old_logprob = propdist.log_prob(next_config)
        next_config.requires_grad_(True)
        next_config_logprobs = trialfunc(next_config)
        next_grads, = torch.autograd.grad(next_config_logprobs,
                                    next_config,
                                    grad_outputs=grad_out,
                                    retain_graph=False)
        with torch.no_grad():
            reverse_propdist = Normal(next_config + eta*next_grads, np.sqrt(2.0*eta))
            old_new_logprob = reverse_propdist.log_prob(old_point)
        return next_config, new_old_logprob, old_new_logprob
    
    return metropolis_asymmetric(trialfunc, mala_proposal, num_walkers=num_walkers, num_steps=num_steps)





In [56]:
def energy_minimize_step(trialfunc, samples, optimizer):
    with torch.no_grad():
        local_energies = trialfunc.local_energy(samples)
    sample_logprobs = trialfunc(samples)
    loss = (local_energies * sample_logprobs).mean()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

$$\nabla \mathbb{E}[f(x)] = \mathbb{E}[f(x) \nabla \log \pi(x)]$$

In [68]:
tf = HarmonicTrialFunction(torch.tensor(1.2))

In [63]:
results = metropolis_symmetric(tf, normal_proposal, num_walkers=100, num_steps=20000)

In [64]:
true_mean_energy(1.2)

1.067222222222222

In [65]:
torch.mean(tf.local_energy(results))

tensor(1.0666, grad_fn=<MeanBackward0>)

In [59]:
results = mala(tf, num_walkers=100, num_steps=20000, eta=0.01)

In [60]:
torch.mean(tf.local_energy(results))

tensor(1.0649, grad_fn=<MeanBackward0>)

In [69]:
def vmc_iterate(tf, num_iters=100):
    opt = optim.SGD(tf.parameters(), lr=1e-2,momentum=0.9)
    for i in range(num_iters):
        results=metropolis_symmetric(tf, normal_proposal, num_walkers=1000, num_steps=5000)
        energy_minimize_step(tf, results, opt)
        print(tf.alpha)

In [70]:
vmc_iterate(tf)

Parameter containing:
tensor(1.1938, requires_grad=True)
Parameter containing:
tensor(1.1822, requires_grad=True)
Parameter containing:
tensor(1.1659, requires_grad=True)
Parameter containing:
tensor(1.1460, requires_grad=True)
Parameter containing:
tensor(1.1232, requires_grad=True)
Parameter containing:
tensor(1.0986, requires_grad=True)
Parameter containing:
tensor(1.0729, requires_grad=True)
Parameter containing:
tensor(1.0473, requires_grad=True)
Parameter containing:
tensor(1.0224, requires_grad=True)
Parameter containing:
tensor(0.9991, requires_grad=True)
Parameter containing:
tensor(0.9782, requires_grad=True)
Parameter containing:
tensor(0.9601, requires_grad=True)
Parameter containing:
tensor(0.9455, requires_grad=True)
Parameter containing:
tensor(0.9348, requires_grad=True)
Parameter containing:
tensor(0.9280, requires_grad=True)
Parameter containing:
tensor(0.9251, requires_grad=True)
Parameter containing:
tensor(0.9259, requires_grad=True)
Parameter containing:
tensor(0.